In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import re
import glob
import math
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta
from collections import defaultdict
from multiprocessing import Manager
import sklearn

In [2]:
spark = (
    SparkSession.builder.appName('Fraud Imputation')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/10/03 04:31:29 WARN Utils: Your hostname, DESKTOP-F216TKE resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/03 04:31:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/03 04:31:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
full_transactions = spark.read.parquet(".././data/curated/transaction_external")

In [4]:
full_transactions.show()

24/10/03 04:31:34 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+----+--------+--------------+------------+-------+-----------+------------------+--------------------+------------------+-----+-----------+-----------------+--------------------+--------------------+----+---------+--------------+----------+--------------+----------------+------------------+-------------------+------------------+-------------------------+-------------------------+----------------------+------+-------+-------+------------+------------+------------+-----------+-----------+------------+-----------+------------+--------------+------------+------------+------------+-----------+-----------+------------+-----------+------------+--------------+
|month|year|postcode|order_datetime|merchant_abn|user_id|consumer_id|      dollar_value|            order_id|     consumer_name|state|     gender|fraud_probability|       merchant_name|                tags|type|take_rate|merchant_fraud|AREASQKM21|    SHAPE_Leng|      SHAPE_Area|               lon|                lat|Median_age_per

In [5]:
full_transactions.printSchema()

root
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- merchant_abn: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- consumer_id: string (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- consumer_name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- fraud_probability: double (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- type: string (nullable = true)
 |-- take_rate: double (nullable = true)
 |-- merchant_fraud: string (nullable = true)
 |-- AREASQKM21: double (nullable = true)
 |-- SHAPE_Leng: double (nullable = true)
 |-- SHAPE_Area: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- Median_age_persons: double (nullable = true)
 |-- M

In [6]:
full_transactions = full_transactions.withColumn('merchant_fraud', F.col('merchant_fraud').cast(DoubleType()))

In [7]:
full_transactions = full_transactions.withColumn('order_time', F.unix_timestamp('order_datetime'))

In [8]:
continuous_features = ["dollar_value", "take_rate", "order_time", "Average_household_size", "Median_age_persons", "Median_tot_fam_inc_weekly", "Median_tot_hhd_inc_weekly", "Tot_M", "Tot_P", "Tot_F"]
categorical_features = ["state", "gender",  "type"]

In [9]:
merchant_fraud = spark.read.parquet(".././data/curated/merchant_fraud")

In [10]:
merchant_fraud.show()

+------------+--------------+------------------+
|merchant_abn|order_datetime| fraud_probability|
+------------+--------------+------------------+
| 19492220327|    2021-11-28|44.403658647495355|
| 31334588839|    2021-10-02| 42.75530083865367|
| 19492220327|    2021-12-22|38.867790051131095|
| 90918180829|    2021-09-02| 43.32551731714902|
| 31334588839|    2021-12-26| 38.36165958070444|
| 31334588839|    2021-11-26| 36.20971272078342|
| 19492220327|    2021-12-18|33.819672154331755|
| 31334588839|    2021-11-29|35.386213297375505|
| 19492220327|    2021-11-18|32.193139919494016|
| 93260930990|    2021-11-30| 37.87197154172081|
| 90918180829|    2021-09-16| 36.62001350882694|
| 83199298021|    2022-02-27|26.025158824861773|
| 83199298021|    2022-02-17| 25.77998392496447|
| 97217894162|    2022-01-21| 34.94582650821017|
| 43083074133|    2021-12-18| 48.66890735193894|
| 19492220327|    2021-10-18|30.935658544112286|
| 31334588839|    2021-09-23|  32.3890854385705|
| 90918180829|    20

In [11]:
merchant_fraud = merchant_fraud.withColumnRenamed("fraud_probability","merchant_fraud")

In [12]:
consumer_fraud = spark.read.parquet(".././data/curated/consumer_fraud")

In [13]:
consumer_fraud.show()

+-------+--------------+-----------------+
|user_id|order_datetime|fraud_probability|
+-------+--------------+-----------------+
|   6228|    2021-12-19| 97.6298077657765|
|  21419|    2021-12-10|99.24738020302328|
|   5606|    2021-10-17|84.05825045251777|
|   3101|    2021-04-17|91.42192091901347|
|  22239|    2021-10-19|94.70342477508035|
|  16556|    2022-02-20|89.65663294494827|
|  10278|    2021-09-28|83.59136689427714|
|  15790|    2021-12-30|71.77065889280253|
|   5233|    2021-08-29|85.87123303878818|
|    230|    2021-08-28|86.28328808934151|
|  13601|    2021-12-26|83.13696487489679|
|   6383|    2021-09-15| 66.2676451623754|
|   3513|    2022-02-27|75.16981192247916|
|  18658|    2021-10-19|82.98609082999361|
|   5965|    2021-11-14|69.37164467869053|
|  18714|    2021-11-14|83.78813794627237|
|  22957|    2022-02-12|82.79065699075498|
|  20118|    2021-09-05|80.34030486265003|
|   6436|    2021-12-24|84.81618344606828|
|  17900|    2022-02-25|92.73262811161372|
+-------+--

In [14]:
full_transaction = full_transactions.na.drop(subset=["merchant_name"])
full_transaction = full_transactions.na.drop(subset=["Median_age_persons"])

training_data = full_transaction.join(merchant_fraud, on=['merchant_abn', 'order_datetime','merchant_fraud'], how='inner')
consumer_training_data = full_transaction.join(consumer_fraud, on=['user_id', 'order_datetime','fraud_probability'], how='inner')


null_merchant_fraud = full_transaction.filter(F.col("merchant_fraud").isNull())
null_consumer_fraud = full_transaction.filter(F.col("fraud_probability").isNull())

full_transaction.show()

+-----+----+--------+--------------+------------+-------+-----------+------------------+--------------------+------------------+-----+-----------+-----------------+--------------------+--------------------+----+---------+--------------+----------+--------------+----------------+------------------+-------------------+------------------+-------------------------+-------------------------+----------------------+------+-------+-------+------------+------------+------------+-----------+-----------+------------+-----------+------------+--------------+------------+------------+------------+-----------+-----------+------------+-----------+------------+--------------+----------+
|month|year|postcode|order_datetime|merchant_abn|user_id|consumer_id|      dollar_value|            order_id|     consumer_name|state|     gender|fraud_probability|       merchant_name|                tags|type|take_rate|merchant_fraud|AREASQKM21|    SHAPE_Leng|      SHAPE_Area|               lon|                lat|Med

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

In [16]:
for i in categorical_features:
    indexer = StringIndexer(inputCol=i, outputCol=f"{i}_index", handleInvalid="keep").fit(training_data)
    training_data = indexer.transform(training_data)
    consumer_training_data = indexer.transform(consumer_training_data)
    null_merchant_fraud = indexer.transform(null_merchant_fraud)
    null_consumer_fraud = indexer.transform(null_consumer_fraud)

for j in categorical_features:
    encoder = OneHotEncoder(inputCol=f"{j}_index", outputCol=f"{j}_vector").fit(training_data)
    training_data = encoder.transform(training_data)
    consumer_training_data = encoder.transform(consumer_training_data)
    null_merchant_fraud = encoder.transform(null_merchant_fraud)
    null_consumer_fraud = encoder.transform(null_consumer_fraud)

assembler = VectorAssembler(
    inputCols=[f"{k}_vector" for k in categorical_features] + continuous_features,
    outputCol="feature_vector",
    handleInvalid="keep"
)

training_data = assembler.transform(training_data)
consumer_training_data = assembler.transform(consumer_training_data)
null_merchant_fraud = assembler.transform(null_merchant_fraud)
null_consumer_fraud = assembler.transform(null_consumer_fraud)

In [17]:
train_data, test_data = training_data.randomSplit([0.8, 0.2])
consumer_train_data, consumer_test_data = consumer_training_data.randomSplit([0.8, 0.2])
LR =LinearRegression(featuresCol='feature_vector', labelCol='merchant_fraud', regParam=0.1)
LR2 =LinearRegression(featuresCol='feature_vector', labelCol='fraud_probability', regParam=0.1)

merchant_LR_model = LR.fit(train_data)
consumer_LR_model = LR2.fit(consumer_train_data)

merchant_results = merchant_LR_model.transform(test_data)
consumer_results = consumer_LR_model.transform(consumer_test_data)

merchant_results.select(["prediction","merchant_fraud"]).show(900)
consumer_results.select(["prediction","fraud_probability"]).show(900)
print("Merchant RMSE:", merchant_LR_model.summary.rootMeanSquaredError)
print("Merchant MAE:", merchant_LR_model.summary.meanAbsoluteError)

print("Consumer RMSE:", consumer_LR_model.summary.rootMeanSquaredError)
print("Consumer MAE:", consumer_LR_model.summary.meanAbsoluteError)

merchant_pandas_results = merchant_results.toPandas()
merchant_mse = mean_squared_error(merchant_pandas_results['prediction'], merchant_pandas_results['merchant_fraud'])
merchant_rmse = np.sqrt(merchant_mse)
print("Manual Merchant RMSE:", merchant_rmse)

consumer_pandas_results = consumer_results.toPandas()
consumer_mse = mean_squared_error(consumer_pandas_results['prediction'], consumer_pandas_results['fraud_probability'])
consumer_rmse = np.sqrt(consumer_mse)
print("Manual Consumer RMSE:", consumer_mse)

+------------------+------------------+
|        prediction|    merchant_fraud|
+------------------+------------------+
| 38.53576014825671|30.761664332047502|
| 42.23226963103252| 23.99859726586608|
| 34.03429549708153|34.341900552182295|
|39.847691519184764|34.341900552182295|
| 34.20782198824213|31.537398474248473|
|  34.0492825595129|31.537398474248473|
| 34.66857120790996|31.537398474248473|
| 40.02929098217783|  67.1177397510102|
| 38.69512944312578|23.920305772666964|
| 38.69512944312578|23.920305772666964|
|42.516272897746376|25.054391991473924|
| 36.81832129568215|25.054391991473924|
| 34.17298696248258| 42.75530083865367|
| 31.64003334266272| 61.92380860960476|
|32.953298625760226|  67.5058112247036|
|32.256530772251836|23.920305772666964|
|33.491672818942334| 72.73069736562613|
|46.919729969549735| 18.21089142894488|
| 38.56528668040096| 43.32551731714902|
| 38.12396067846953| 36.62001350882694|
|62.857686710441044| 63.37734364737917|
|29.216001749823363|29.102418916409913|


+------------------+------------------+
|        prediction| fraud_probability|
+------------------+------------------+
|   15.234293403246| 9.816453160367068|
|12.276404063324435| 9.933529126365185|
| 18.37165807676962| 17.68193465360408|
|12.823328138676573| 17.68193465360408|
|17.149924676551535|10.864447214399984|
| 12.21487965609981| 31.35531016213912|
|11.088884569928922|14.612214853430679|
|17.232292871395288|12.584454460100943|
|15.806836819771632|  9.60971403539899|
|15.636413428634938|10.884406392357496|
| 17.12450100615439|17.755982014591375|
|19.731610715820636|18.691682927762375|
|13.187640615770022|13.331029073132958|
| 15.92382740113382|  9.45388755844229|
|13.567090537046766|31.260338994577967|
| 17.76000327626042|15.303177414593677|
|12.481883629959071|  8.59896476074255|
|12.151639472527108|  8.60091119260379|
|13.483264557684322|13.372665599121916|
|15.212176718656025| 8.528463744809839|
|27.554829036166268|39.878177015326145|
|13.427963028799297|10.838943225720927|


Manual Merchant RMSE: 5.024767402780939


Manual Consumer RMSE: 87.09382275776493


In [18]:
LR =LinearRegression(featuresCol='feature_vector', labelCol='merchant_fraud', regParam=0.1)
LR_model = LR.fit(training_data)
null_values = LR_model.transform(null_merchant_fraud)

In [19]:
LR2 =LinearRegression(featuresCol='feature_vector', labelCol='fraud_probability', regParam=0.1)
LR2_model = LR2.fit(consumer_training_data)
consumer_null_values = LR2_model.transform(null_consumer_fraud)

In [20]:
null_values = null_values.select("order_id","prediction")
null_values = null_values.withColumnRenamed("prediction","predicted_merchant_fraud_probability")
full_transaction = full_transaction.join(null_values, on="order_id", how = "left")
full_transaction = full_transaction.withColumn("merchant_fraud", F.when(F.col("merchant_fraud").isNull(), 
                                                                                      F.col("predicted_merchant_fraud_probability"))
                                                                                      .otherwise(F.col("merchant_fraud")))
full_transaction = full_transaction.drop("predicted_merchant_fraud_probability")

In [21]:
consumer_null_values = consumer_null_values.select("order_id","prediction")
consumer_null_values = consumer_null_values.withColumnRenamed("prediction","predicted_consumer_fraud_probability")
full_transaction = full_transaction.join(consumer_null_values, on="order_id", how = "left")
full_transaction = full_transaction.withColumn("fraud_probability", F.when(F.col("fraud_probability").isNull(), 
                                                                                      F.col("predicted_consumer_fraud_probability"))
                                                                                      .otherwise(F.col("fraud_probability")))
full_transaction = full_transaction.drop("predicted_consumer_fraud_probability")

In [22]:
full_transaction.show()

+--------------------+-----+----+--------+--------------+------------+-------+-----------+------------------+------------------+-----+-----------+------------------+--------------------+--------------------+----+---------+------------------+----------+--------------+----------------+------------------+-------------------+------------------+-------------------------+-------------------------+----------------------+-------+-------+-------+------------+------------+------------+-----------+-----------+------------+-----------+------------+--------------+------------+------------+------------+-----------+-----------+------------+-----------+------------+--------------+----------+
|            order_id|month|year|postcode|order_datetime|merchant_abn|user_id|consumer_id|      dollar_value|     consumer_name|state|     gender| fraud_probability|       merchant_name|                tags|type|take_rate|    merchant_fraud|AREASQKM21|    SHAPE_Leng|      SHAPE_Area|               lon|            

In [24]:
full_transaction.write.mode('overwrite').parquet(".././data/curated/final_transactions")

24/10/03 04:39:32 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.
24/10/03 04:39:35 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.
24/10/03 04:40:16 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.
24/10/03 04:40:16 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.
24/10/03 04:40:17 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not imme